In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch

base = 'chavinlo/alpaca-native'
model = AutoModelForCausalLM.from_pretrained(
    base,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    device_map="auto"
)

local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    
    model=model,
    
    tokenizer=base,
    
    max_length=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
    # eos_token_id=2,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))
    


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
ask('What is the capital of England?')

{'intruction': 'What is the capital of England?', 'text': ' London'}


In [3]:
ask('내 이름이 뭘까?')

{'intruction': '내 이름이 뭘까?', 'text': 'Your name is...'}


In [4]:
chat('나의 이름은 전재형이야')
chat('니 이름은 뭐니?')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:

질문: 나의 이름은 전재형이야
답변:

> Finished chain.
 하는 것을 제시합니다.


> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  하는 것을 제시합니다.
질문: 니 이름은 뭐니?
답변:

> Finished chain.
 어떤 이름인가요?


In [5]:
chat('내 이름은 뭘까??')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  하는 것을 제시합니다.
Human: 니 이름은 뭐니?
AI:  어떤 이름인가요?
질문: 내 이름은 뭘까??
답변:

> Finished chain.
 That's a tough one!
